# Building a RBF Network for classification tasks

In [6]:
import numpy as np
import pandas as pd

In [7]:
data=pd.read_excel('iris.xlsx').values
from nnfs.datasets import spiral_data
#X,Y=spiral_data(500,3)
X=data[:,2:4]
Y=data[:,5:8]
j=data[:,-1]

In [8]:
class RBF_network:
    
    def __init__(self,X,Y,K):
        self.X=X
        self.Y=Y
        self.K=K
        self.mu=None
        self.sigma=None
        self.RBS_X=None
        self.W=None
    
    def k_means(self,iters,epsilon):

        #Initial variables
        mu=X[np.random.choice(range(len(self.X)), self.K, replace=False)]
        algho_converged=False
        current_it=0

        while(not algho_converged) and (current_it<iters):

            #Initialize new clusters 
            clust=[[] for i in range(len(mu))]

            #Calculate distances
            for i in self.X:
                dist=[]
                for j in mu:
                    dist.append(np.sqrt(sum((i-j)**2)))
                #Assign clusters from distances
                clust[int(np.argmin(dist))].append(i)

            #Save previous clusters
            mu_prev=mu.copy()

            #Recalculate centroids (mu)
            for i in range(len(clust)):
                mu[i]=np.mean(clust[i],axis=0)

            #Calculate if series has converged
            clust_change=np.sum((mu_prev-mu)**2)
            algho_converged=(-epsilon<clust_change<epsilon)

            #Sum up new iteration
            current_it+=1

        #Standard deviation from clusters
        sigma=np.zeros((len(clust)))
        for i in range(len(clust)):
            sigma[i]=np.std(clust[i])

        #Update class parameters
        self.mu,self.sigma=mu,sigma
        
    #Function to calculate the gaussian norm
    def gaussian_rbf(self,x,m,s):
        norm=np.sqrt(sum((x-m)**2))
        return np.exp(-0.5*(norm**2)/(s**2))
    
    def fit(self):
        #RBF transformation to Xs
        RBF_X=[]
        for x in self.X:
            RBF_X.append([self.gaussian_rbf(x,m,s) for (m,s) in zip(self.mu,self.sigma)])
        self.RBF_X=np.array(RBF_X)
        #OLS solution
        weights=np.dot(np.linalg.pinv(np.dot(self.RBF_X.T,self.RBF_X)), np.dot(self.RBF_X.T,self.Y))
        self.W=weights
    
    def predict(self,X_test):
        #RBF transformation to X test
        RBF_Xtest=[]
        for x in X_test:
            RBF_Xtest.append([self.gaussian_rbf(x,m,s) for (m,s) in zip(self.mu,self.sigma)])
        
        #Predictions
        preds=[np.argmax(i)+1 for i in np.dot(np.array(RBF_Xtest),self.W)]
        
        return preds

In [11]:
nn=RBF_network(X,Y,5)
nn.k_means(1000,0.00001)
nn.fit()
Y_p=nn.predict(X)

#Accuracy
diff=j-Y_p
len(diff[diff==0])/len(diff)

0.96